In [28]:
import numpy as np
from PIL import Image
from glob import glob # manipulação de lista de nomes de arquivos
from skimage.measure import regionprops # propriedades de cada segmento
import matplotlib.pyplot as plt
from scipy import ndimage

In [30]:
# Célula com funções para investigar se um segmento é misto, obter HoR e obter a classe majoritária de um segmento

# 0 - Fundo
# 1 - Desmatamento recente
# 2 - Floresta
# 3 - Ñ analisado

def is_mixed(segment):
    # flattening segment
    segment = segment.flatten()


    NFP = np.count_nonzero(segment == 2) # se floresta
    NP = np.count_nonzero(segment) # desconsiderando o fundo (np.zeros)
    NNP = NP - NFP # Calcula os pixels de não-fundo

    if NFP != 0 and NNP != 0: # se em um segmento tiver pixels de desmata e floresta, ele é misto
        return True
    
    return False
    

def get_hor(segment):
    
    # flattening segment (transformando a matriz 'segment' em um array unidimensional)
    segment = segment.flatten()

    # Number Foreground Pixels: conta o número de pixels no vetor segment que são diferentes de zero. Especificamente, os que têm o valor 2
    NFP = np.count_nonzero(segment == 2)

    # conta o número total de pixels no vetor 'segment' que são diferentes de zero. Isso representa o número total de pixels
    NP = np.count_nonzero(segment) # desconsiderando o fundo
    
    # Número de pixels de não-fundo: subtração da quantidade de pixels total de não fundo pela quantidade de pixels iguais a 170
    NNP = NP - NFP

    HoR = max([NFP, NNP]) / NP

    return HoR


def get_major_class(segment):
    if np.argmax(np.bincount(segment.flatten())) == 2:
        return "forest"
    elif np.argmax(np.bincount(segment.flatten())) == 1:
        return "non forest"
    elif np.argmax(np.bincount(segment.flatten())) == 3:
        return "not analyzed"
    else:
        return np.argmax(np.bincount(segment.flatten()))
    
def calculate_statistics(segmentation):
    unique_classes, counts = np.unique(segmentation, return_counts=True)
    std_dev = np.std(counts)
    mean = np.mean(counts)
    coeff = round(std_dev/mean, 4)

    return std_dev, mean, coeff

In [5]:
grayscale = np.load('/home/jp/Documents/IME/TCC/NEW/saved_segmentations/area3_despeckle_maskslic_10000.npy')
print(grayscale.shape)
a, b, c = calculate_statistics(grayscale)
a = round(a, 4)
b = round(b,4)
c = round(c,4)
print(f'{a} & {b} & {c}')

(2132, 2093)
18579.3457 & 451.0083 & 41.1951


In [6]:
# Construir as listas de arquivos a processar
base = 'data/'
slic_path = base + 'saved_slic/'
truth_path = base + 'saved_truth/'
file_list = [
    'ae03_sentinel_rgb.npy',
    'sentinel_rgb_mask.npy',
    'AREA3_c_10000_hls_mask_corrigido.npy', 
    'AREA3_c_10000_rw_mask_corrigido.npy', 
    'AREA3_c_10000_snll_mask_corrigido.npy', 
    'AREA3_c_10000_wishart_mask_corrigido.npy']
sentinel_truth = 'ae03_truth.npy'
sentinel_big_truth = 'A3_big.npy'

parameters = {
    0: {
        'name': 'Sentinel-2',
        'truth': truth_path + sentinel_big_truth,
        'slic': slic_path + 'ae03_sentinel_rgb.npy'
    },
    1: {
        'name': 'Sentinel-2 masked',
        'truth': truth_path + sentinel_big_truth,
        'slic': slic_path + 'sentinel_rgb_mask.npy'
    },
    2: {
        'name': 'Wishart mask',
        'truth': truth_path + sentinel_truth,
        'slic': slic_path + 'AREA3_c_10000_wishart_mask_corrigido.npy'
    },
    3: {
        'name': 'RW mask',
        'truth': truth_path + sentinel_truth,
        'slic': slic_path + 'AREA3_c_10000_rw_mask_corrigido.npy'
    },
    4: {
        'name': 'SNLL mask',
        'truth': truth_path + sentinel_truth,
        'slic': slic_path + 'AREA3_c_10000_snll_mask_corrigido.npy'
    },
    5: {
        'name': 'HLT mask',
        'truth': truth_path + sentinel_truth,
        'slic': slic_path + 'AREA3_c_10000_hls_mask_corrigido.npy'
    }
}

In [20]:
# parameters = {
#     0: {
#         'name': "Grayscale mask",
#         'truth': '/home/jp/Pictures/new_truth/ae03_truth.npy',
#         'slic': '/home/jp/Documents/IME/TCC/NEW/saved_segmentations/area3_despeckle_maskslic_10000.npy'
#     }
# }

parameters = {
    0: {
        'name': "Colorized mask",
        'truth': '/home/jp/Pictures/new_truth/ae03_truth.npy',
        'slic': '/home/jp/Documents/IME/TCC/NEW/saved_segmentations/area3_colorized_maskslic_10000.npy'
    }
}

In [33]:
# Construir as listas de arquivos a processar
base = '/home/jp/Documents/IME/TCC/CODE/Capstone-Project/data/'
slic_path = base + 'slic/'
truth_path = base + 'truth/'

for area in ['area2', 'area4']:
    print(area)
    s1_truth = truth_path + area + '_s1.npy'
    s2_truth = truth_path + area + '_s2.npy'
    color_truth = truth_path + area + '_color.npy'

    area_path = slic_path + area + '/' + area + '_'
    
    for option in ['', '_mask']:

        parameters = {
            0: {
                'name': 'Sentinel-2',
                'truth': s2_truth,
                'slic': area_path + 's2' + option + '.npy'
            },
            1: {
                'name': 'Wishart',
                'truth': s1_truth,
                'slic': area_path + 'wishart' + option + '.npy'
            },
            2: {
                'name': 'RW',
                'truth': s1_truth,
                'slic': area_path + 'rw' + option + '.npy'
            },
            3: {
                'name': 'SNLL',
                'truth': s1_truth,
                'slic': area_path + 'snll' + option + '.npy'
            },
            4: {
                'name': 'HLT',
                'truth': s1_truth,
                'slic': area_path + 'hlt' + option + '.npy'
            },
            5: {
                'name': 'speckle2void',
                'truth': s1_truth,
                'slic': area_path + 'despeckle' + option + '.npy'
            },
            6: {
                'name': 'cGAN4ColSAR',
                'truth': color_truth,
                'slic': area_path + 'colorized' + option + '.npy'
            },
        }

        parameters = {
            0:  {
                'name': 'speckle2void',
                'truth': s1_truth,
                'slic': area_path + 'despeckle' + option + '.npy'
            }
        }


        for idx in parameters.keys(): # para cada uma das imagens (ou slics)
            total_segs = 0 
            good_segs = 0
            bad_by_hor = 0
            bad_by_size = 0
            n_perfect_segs = 0 # Quando HoR = 1
            truth = np.load(parameters[idx].get('truth')) # Carrega a verdade da idx-ésima imagem (truth_x01.npy, por exemplo)
            slic = np.load(parameters[idx].get('slic')) # Carrega o slic da idx-ésima imagem (slic_x01.npy, por exemplo)

            # if parameters[idx].get('name') == 'speckle2void':

            #     target_shape = (truth.shape[0], truth.shape[1])

            #     # Resize the array using SciPy's resize function
            #     slic = ndimage.zoom(slic, 
            #                                 (target_shape[0] / slic.shape[0], target_shape[1] / slic.shape[1]), 
            #                                 order=3) 
            
            # target_shape = (2990, 2933, 1)

        
            # mask = truth
            # print(mask.shape)
            # # Resize the array using SciPy's resize function
            # mask = ndimage.zoom(mask, 
            #                             (target_shape[0] / mask.shape[0], target_shape[1] / mask.shape[1], 1), 
            #                             order=3) 
            # print(mask.shape)
            # # mask = mask[mask.shape[0]-2048:mask.shape[0],mask.shape[1]-2048:mask.shape[1]]
            # truth = mask[mask.shape[0]-2048:mask.shape[0],mask.shape[1]-2048:mask.shape[1]]

            props = regionprops(slic) # Calcula as propriedades de cada segmento e atribui a 'props'
            segments_truth = [] # List com os segmentos da verdade
            segments_image = [] # List com os segmentos da imagem
            bad_segment_ids = [] # List com os ids dos segmentos ruins
            perfect_segs_truth = [] # List com os segmentos perfeitos da verdade

            sd, mean, coeff = calculate_statistics(slic)

            assert slic.shape[:2] == truth.shape[:2] # Checa se o slic corrente possui o mesmo shape (h, w) da verdade. Caso contrário, a execução é interrompida
            
            for i, prop in enumerate(props): # para cada segmento i do slic idx...
                
                total_segs += 1 # count...
                
                minr, minc, maxr, maxc = prop.bbox # minr: linha superior, minc: coluna mais à esquerda, maxr: linha inferior, maxc: coluna mais à direita

                # As linhas a seguir criam estruturas (arrays) vazias para os segmentos da verdade e da imagem
                segment_truth = np.zeros((maxr - minr, maxc - minc)) # delimita o tamanho do segmento da verdade
                coords = np.array(prop.coords) # pega as coordenadas de cada pixel

                for pixel in coords: # para cada coordenada de pixel dentro do i-ésimo segmento
                    segment_truth[pixel[0] - minr, pixel[1] - minc] = truth[pixel[0], pixel[1]] # faz uma cópia, pixel a pixel do i-ésimo segmento da verdade

                hor = get_hor(segment_truth) # HoR da verdade
                classification = get_major_class(np.uint8(segment_truth))

                # se a quantidade de pixels no i-ésimo segmento na verdade > 70, HoR desse segmento > 0.7 e ele é floresta ou não floresta: adicione o 
                if (segment_truth.shape[0] * segment_truth.shape[1] > 70) and (get_hor(segment_truth) > 0.7) and (classification in ["forest", "non forest"]):
                    segments_truth.append(np.uint8(segment_truth))             
                    good_segs += 1

            # print(f'{parameters[idx].get('name')}: total_segs = {total_segs} | good_segs = {good_segs} | good%: {round(good_segs/total_segs, 4)}')
            latex_string = f'& {total_segs} & {good_segs} & {round(good_segs/total_segs, 4)} & {mean} & {sd} & {coeff} '
            print(latex_string) 


area2
& 9804 & 0 & 0.0 & 579.2047812817905 & 81.22012387585313 & 0.1402 
& 9896 & 9378 & 0.9477 & 575.283722340103 & 30208.82468382952 & 52.5112 
area4
& 9564 & 0 & 0.0 & 290.2741582403836 & 43.44106021838919 & 0.1497 
& 9829 & 9012 & 0.9169 & 283.2756866734486 & 20032.705618331525 & 70.7181 


In [27]:
global_segment_array = [] # segs
global_perfect_segments = [] # perfect segs
from scipy import ndimage

for idx in parameters.keys(): # para cada uma das imagens (ou slics)
    total_segs = 0 
    good_segs = 0
    bad_by_hor = 0
    bad_by_size = 0
    n_perfect_segs = 0 # Quando HoR = 1
    truth = np.load(parameters[idx].get('truth')) # Carrega a verdade da idx-ésima imagem (truth_x01.npy, por exemplo)
    slic = np.load(parameters[idx].get('slic')) # Carrega o slic da idx-ésima imagem (slic_x01.npy, por exemplo)

    target_shape = (2990, 2933, 1)

 
    mask = truth
    print(mask.shape)
    # Resize the array using SciPy's resize function
    mask = ndimage.zoom(mask, 
                                (target_shape[0] / mask.shape[0], target_shape[1] / mask.shape[1], 1), 
                                order=3) 
    print(mask.shape)
    # mask = mask[mask.shape[0]-2048:mask.shape[0],mask.shape[1]-2048:mask.shape[1]]
    truth = mask[mask.shape[0]-2048:mask.shape[0],mask.shape[1]-2048:mask.shape[1]]

    props = regionprops(slic) # Calcula as propriedades de cada segmento e atribui a 'props'
    segments_truth = [] # List com os segmentos da verdade
    segments_image = [] # List com os segmentos da imagem
    bad_segment_ids = [] # List com os ids dos segmentos ruins
    perfect_segs_truth = [] # List com os segmentos perfeitos da verdade

    sd, mean, coeff = calculate_statistics(slic)

    assert slic.shape[:2] == truth.shape[:2] # Checa se o slic corrente possui o mesmo shape (h, w) da verdade. Caso contrário, a execução é interrompida
    
    for i, prop in enumerate(props): # para cada segmento i do slic idx...
        
        total_segs += 1 # count...
        
        minr, minc, maxr, maxc = prop.bbox # minr: linha superior, minc: coluna mais à esquerda, maxr: linha inferior, maxc: coluna mais à direita

        # As linhas a seguir criam estruturas (arrays) vazias para os segmentos da verdade e da imagem
        segment_truth = np.zeros((maxr - minr, maxc - minc)) # delimita o tamanho do segmento da verdade
        coords = np.array(prop.coords) # pega as coordenadas de cada pixel

        for pixel in coords: # para cada coordenada de pixel dentro do i-ésimo segmento
            segment_truth[pixel[0] - minr, pixel[1] - minc] = truth[pixel[0], pixel[1]] # faz uma cópia, pixel a pixel do i-ésimo segmento da verdade

        hor = get_hor(segment_truth) # HoR da verdade
        classification = get_major_class(np.uint8(segment_truth))

        # se a quantidade de pixels no i-ésimo segmento na verdade > 70, HoR desse segmento > 0.7 e ele é floresta ou não floresta: adicione o 
        if (segment_truth.shape[0] * segment_truth.shape[1] > 70) and (get_hor(segment_truth) > 0.7) and (classification in ["forest", "non forest"]):
            segments_truth.append(np.uint8(segment_truth))             
            good_segs += 1

    # print(f'{parameters[idx].get('name')}: total_segs = {total_segs} | good_segs = {good_segs} | good%: {round(good_segs/total_segs, 4)}')
    latex_string = f'& {total_segs} & {good_segs} & {round(good_segs/total_segs, 4)} & {mean} & {sd} & {coeff} '
    print(latex_string) 

(2132, 2093, 1)
(2990, 2933, 1)
& 8688 & 5724 & 0.6588 & 482.714236390839 & 26695.58176104298 & 55.3031 
